The goal is to pass in a list of different theta and range (of errors), and see the differences between outcomes. Using Monte Carlo simulation.

This is a version that generates only a list of outcomes with mean theta.

#### Agent Runner

In [ ]:
from mTree.microeconomic_system.agent import Agent
from mTree.microeconomic_system.directive_decorators import *
import random
import numpy as np
from mTree.microeconomic_system.message import Message
from mTree.microeconomic_system.container import Container
from basic_agent import BasicAgent
from basic_envrionment import BasicEnvironment
from basic_institution import BasicInstitution

from types import FunctionType

bundles = []
length = 10
for i in range(1, length + 1):
    optionA = ([((i / length), 2), ((((length - i) / length)), 1.6),])
    optionB = ([((i / length),3.85), ((((length - i) / length)), 0.10)])
    bundles.append([optionA, optionB])  # each row of the lottery


container = Container()
container.create_root_environment(BasicEnvironment)
container.setup_environment_institution(BasicInstitution)
container.setup_environment_agents(BasicAgent, 1)

message = Message() # Initialize the enviorment
message.set_sender("external")
message.set_directive("initialize_environment")
payload = {}
payload["get_theta"] = 0.5
payload["get_sd"] = 0.15
payload["get_bundles"] = bundles
message.set_payload(payload)
container.send_root_environment_message(message)


message = Message()
message.set_sender("external")
message.set_directive("fill_in_agents")
container.send_root_environment_message(message)


#### Basic Environment

In [ ]:
from mTree.microeconomic_system.agent import Agent
from mTree.microeconomic_system.environment import Environment

from basic_agent import BasicAgent
from basic_institution import BasicInstitution

import numpy as np
from mTree.microeconomic_system.message_space import MessageSpace
from mTree.microeconomic_system.message import Message
from mTree.microeconomic_system.directive_decorators import *
import random

@directive_enabled_class
class BasicEnvironment(Environment):
    def __init__(self):
        print('environment defined')
        self.theta = None
        self.sd = None
        self.bundles = []
        self.agents = []
        self.institutions = []


    @directive_decorator("initialize_environment")
    def ini_env(self, message: Message):
        payload = message.get_payload()

        self.theta = payload["get_theta"]
        self.sd  = payload["get_sd"]
        self.bundles = payload["get_bundles"]
        print(self.theta, self.sd, self.bundles)


    @directive_decorator("fill_in_agents")
    def fill_in_agent(self, message: Message):
        new_message = Message()
        new_message.set_sender(self)
        payload = {}
        payload["get_theta"]  =self.theta
        payload["get_sd"] = self.sd
        payload["get_bundles"] = self.bundles

        print(payload)
        new_message.set_payload(payload)
        new_message.set_directive("agents")
        self.send(self.agents[0], new_message)


#### Basic Agent

In [ ]:
from mTree.microeconomic_system.agent import Agent
from mTree.microeconomic_system.directive_decorators import *
import numpy as np
from mTree.microeconomic_system.message_space import Message
from mTree.microeconomic_system.container import Container
import random


import numpy as np
import sympy as sp
from mTree.microeconomic_system.agent import Agent
from mTree.microeconomic_system.directive_decorators import *
import numpy as np
import random as rnd
from mTree.microeconomic_system.message_space import Message


def CRRA(value, theta):

    if theta == 1.0:
        if value == 0.0:  # Handle value = 0.0
            return 0.0
        else:
            return mat.log(value)
    elif theta > 0:
        if value == 0.0:  # Handle value = 0.0
            return 0.0
        else:
            return (1.0 / (1.0 - theta)) * (value) ** (1.0 - theta)


@directive_enabled_class
class BasicAgent(Agent):
    def __init__(self):
        print("A agent has been created")
        self.theta = None
        self.sd = None
        self.bundles = []
        self.choices = []
        self.outcomes = []


    @directive_decorator("agents")
    def fill_in_agents(self, message: Message):
        print("filling in agents...")
        payload = message.get_payload()
        print(payload)
        self.theta = payload["get_theta"]
        self.sd  = payload["get_sd"]
        self.bundles = payload["get_bundles"]


        for i in range(10):

            row = self.bundles[i]

            sd = self.sd
            get_theta = self.theta
            error = np.random.triangular(-sd, 0 , sd)
            theta = get_theta + error
            print(theta)

            OptionA = row[0]
            OptionB = row[1]

            if len(OptionA) != len(OptionB):  # ensure that there is no error in the row data
                raise ValueError("Length of options must be the same")

                # the probability and utility of OptionA Choice 1
            prob_a1 = (OptionA[0][0])
            out_a1 = CRRA(OptionA[0][1], theta)

            # the probability and utility OptionA Choice 2
            prob_a2 = OptionA[1][0]
            out_a2 = CRRA(OptionA[1][1], theta)

            # the probability and utility OptionB Choice 1
            prob_b1 = (OptionB[0][0])
            out_b1 = CRRA(OptionB[0][1], theta)

            # the probability and utility OptionB Choice 2
            prob_b2 = OptionB[1][0]
            out_b2 = CRRA(OptionB[1][1], theta)

            # expected utility of OptionA and OptionB
            util_a = prob_a1 * out_a1 + prob_a2 * out_a2
            util_b = prob_b1 * out_b1 + prob_b2 * out_b2

            # selects and records the option with larger expected utility
            if util_a > util_b:
                self.choices.append(OptionA)
                self.outcomes.append("A")

            else:
                self.choices.append(OptionB)
                self.outcomes.append("B")

        print(self.choices)
        print(self.outcomes)


#### Basic Institution

In [ ]:
### not used

from mTree.microeconomic_system.institution import Institution
from mTree.microeconomic_system.message_space import Message
from mTree.microeconomic_system.directive_decorators import *
import random as rnd


@directive_enabled_class
class BasicInstitution(Institution):
    def __init__(self):
        print("institution defined")
